# To Produce Small Data File

In [1]:
import os
import sys
sys.path.append("./src")
import h5py
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from das_util import compute_misfit, get_tstamp
from scipy.ndimage import gaussian_filter1d, gaussian_filter
from scipy.interpolate import interp1d

matplotlib.rcParams.update({'font.size': 16})

data_dir='../../data_farm/data/'

psd_data = os.path.join(data_dir, 'psd_all.hdf5')
acf_data = os.path.join(data_dir, 'autocorr_15_60Hz.hdf5')
acf_data_stretched = os.path.join(data_dir, 'autocorr_15_60Hz_stretched.hdf5')
acf_data_stretchedx2 = os.path.join(data_dir, 'autocorr_15_60Hz_stretchedx2.hdf5')
acf_data_stretchedx3 = os.path.join(data_dir, 'autocorr_15_60Hz_stretchedx3.hdf5')
harper_met = os.path.join(data_dir, 'NewportSalop_merged.csv')
reg_met = os.path.join(data_dir, 'met_newport.csv')
spatial_data = os.path.join(data_dir, 'interp_dv_tillage_tire.csv')
phy_data = os.path.join(data_dir, 'Soil_phy.csv')
phy_data_interpolate = os.path.join(data_dir, 'interpolated_soil_phy.csv')

freqmin=25
freqmax=50
samp_freq = 500
dchan = 3.19

## ACFs (H5)
Raw, strected, stretchedx2, stretchedx3 

In [11]:
### Read big h5 files
with h5py.File(acf_data, 'r') as f:
    corr_all_time_ch18 = f['autocorr'][18, :, 2500:3000, 0]
    corr_all_time_ch33 = f['autocorr'][33, :, 2500:3000, 0]
    corr_all_time_ch44 = f['autocorr'][44, :, 2500:3000, 0]
    corr_all_channel_30_40min = f['autocorr'][:50, 30:40, 2500:2700, 0]
with h5py.File(acf_data_stretched, 'r') as f:
    corr_all_time_stretched1_ch18 = f['autocorr'][18, :, :500]
    corr_all_time_stretched1_ch33 = f['autocorr'][33, :, :500]
    corr_all_time_stretched1_ch44 = f['autocorr'][44, :, :500]
with h5py.File(acf_data_stretchedx2, 'r') as f:
    corr_all_time_stretched2_ch18 = f['autocorr'][18, :, :500]
    corr_all_time_stretched2_ch33 = f['autocorr'][33, :, :500]
    corr_all_time_stretched2_ch44 = f['autocorr'][44, :, :500]
with h5py.File(acf_data_stretchedx3, 'r') as f:
    corr_all_time_stretched3_ch18 = f['autocorr'][18, :, :500]
    corr_all_time_stretched3_ch33 = f['autocorr'][33, :, :500]
    corr_all_time_stretched3_ch44 = f['autocorr'][44, :, :500]

### Save small h5 files
with h5py.File(os.path.join(data_dir, 'autocorr_15_60Hz_3chs500pts.hdf5'), 'w') as f:
    f.create_dataset('corr_all_time_ch18', data=corr_all_time_ch18)
    f.create_dataset('corr_all_time_ch33', data=corr_all_time_ch33)
    f.create_dataset('corr_all_time_ch44', data=corr_all_time_ch44)
    f.create_dataset('corr_all_time_stretched1_ch18', data=corr_all_time_stretched1_ch18)
    f.create_dataset('corr_all_time_stretched1_ch33', data=corr_all_time_stretched1_ch33)
    f.create_dataset('corr_all_time_stretched1_ch44', data=corr_all_time_stretched1_ch44)
    f.create_dataset('corr_all_time_stretched2_ch18', data=corr_all_time_stretched2_ch18)
    f.create_dataset('corr_all_time_stretched2_ch33', data=corr_all_time_stretched2_ch33)
    f.create_dataset('corr_all_time_stretched2_ch44', data=corr_all_time_stretched2_ch44)
    f.create_dataset('corr_all_time_stretched3_ch18', data=corr_all_time_stretched3_ch18)
    f.create_dataset('corr_all_time_stretched3_ch33', data=corr_all_time_stretched3_ch33)
    f.create_dataset('corr_all_time_stretched3_ch44', data=corr_all_time_stretched3_ch44)

with h5py.File(os.path.join(data_dir, 'autocorr_15_60Hz_30_40min.hdf5'), 'w') as f:
    f.create_dataset('corr_all_channel_30_40min', data=corr_all_channel_30_40min)


## dVV (H5)

In [12]:
### Iteration 1
with h5py.File(os.path.join(data_dir, 'final_peaks_interp_smooth_15_60Hz.h5'), 'r') as f:
    stretch_ratio = f['final_peaks'][:]

## de ratio using the humidist time period
deratio_dvv = stretch_ratio / np.repeat(np.mean(stretch_ratio[:, 269:280], axis=1)[:, np.newaxis], 482, axis=1) - 1

with h5py.File(os.path.join(data_dir, 'final_peaks_deRatio.h5'), 'w') as f:
    f.create_dataset('deratio_dvv', data=deratio_dvv)


### Iteration 2
with h5py.File(os.path.join(data_dir, 'final_peaks_15_60Hz_2nd_iteration.h5'), 'r') as f:
    stretch_ratio = f['final_peaks'][:50]
stretch_ratio[stretch_ratio<0.75] = 0.75

## de ratio using the humidist time period
stretch_ratio = stretch_ratio / np.repeat(np.mean(stretch_ratio[:, 269:280], axis=1)[:, np.newaxis], 482, axis=1)

stretch_ratio[np.isnan(stretch_ratio)] = 1

deratio_dvv = (deratio_dvv + 1) * stretch_ratio - 1

with h5py.File(os.path.join(data_dir, 'final_peaks_deRatio_2iterations.h5'), 'w') as f:
    f.create_dataset('deratio_dvv', data=deratio_dvv)


### Iteration 3
with h5py.File(os.path.join(data_dir, 'final_peaks_15_60Hz_3rd_iteration.h5'), 'r') as f:
    stretch_ratio = f['final_peaks'][:50]-0.1

## de ratio using the humidist time period
stretch_ratio = stretch_ratio / np.repeat(np.mean(stretch_ratio[:, 269:280], axis=1)[:, np.newaxis], 482, axis=1)

stretch_ratio[np.isnan(stretch_ratio)] = 1

deratio_dvv = (deratio_dvv + 1) * stretch_ratio - 1

with h5py.File(os.path.join(data_dir, 'final_peaks_deRatio_3iterations.h5'), 'w') as f:
    f.create_dataset('deratio_dvv', data=deratio_dvv)

## Stretched ACFs

In [ ]:
# all_ratio = deratio_dvv + 1

# shape_3d = (corr_all_time.shape[0], corr_all_time.shape[1], int(corr_all_time.shape[2]*all_ratio.max())+1)
# corr_all_time_stretched =  np.zeros(shape_3d, dtype=np.float32)

# for iloc in range(50):
#     for itime in range(482):
#         ratio = all_ratio[iloc, itime]
#         tmp = zoom(corr_all_time[iloc, itime], ratio, order=1)
#         corr_all_time_stretched[iloc, itime, :len(tmp)] = tmp 
    
    # plt.figure(figsize = (12, 5))
    # plt.imshow(corr_all_time_stretched[iloc, :, :500].T, aspect='auto', vmin=-0.2, vmax=0.2, cmap='RdBu', origin='lower')

    # plt.xlabel("ACF time (x 5 minutes)", fontsize = 16)
    # plt.ylabel("Time lag (sec)", fontsize = 16)
    # bar = plt.colorbar()
    # bar.set_label('Auto-correlation Coefficient', fontsize = 15)

### stack strected ACF for a static GF
# mean_acf = np.mean(corr_all_time_stretched, axis=1)


# ### Save the stretched ACF
# with h5py.File('data/autocorr_15_60Hz_stretchedx3.hdf5', 'w') as f:
#     f.create_dataset('autocorr', data=corr_all_time_stretched)
#     f.create_dataset('mean_acf', data=mean_acf)